<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon_Anmado-Is./blob/main/Cervus-nippon_reconstruct_GF-SG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Earth Engine Python API 모듈 및 라이브러리 호출
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from osgeo import gdal
import requests
import json
import geopandas as gpd
import os
import shutil
import psutil
from google.colab import drive
import zipfile

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91')

In [2]:
# 데이터 소스 및 기본 설정
mod_sr = ee.ImageCollection("MODIS/006/MOD09Q1")
landsat_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
# Subarea = ee.FeatureCollection("users/Yang_Chen/CIA_subregion")  # 필요한 경우 사용

# 안마도의 위치를 정의하고 관심 영역(AOI)를 생성합니다.
anmado_location = ee.Geometry.Point([126.028111, 35.344031])
aoi = anmado_location.buffer(5000).bounds()  # 안마도를 중심으로 5km 버퍼 생성

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD09Q1! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09Q1

  warnings.warn(warning, category=DeprecationWarning)


In [3]:
#-------------------------------------------------------------------------
# Gap Filling 및 Savitzky–Golay 필터링 방법(GF-SG)을 위한 GEE Python 코드
# 이 스크립트는 8일 간격의 고품질 Landsat NDVI 시계열을 재구성합니다.
#-------------------------------------------------------------------------

# 비트 패킹된 플래그 추출 함수
def getQABits(image, start, end):
    pattern = 0
    for i in range(start, end + 1):
        pattern += 1 << i
    # 추출된 QA 비트의 단일 밴드 이미지 반환
    return image.bitwiseAnd(pattern).rightShift(start)

# 메인 절차
begin_date = '2014-01-01'  # 처리 시작 날짜
end_date = '2015-01-01'    # 처리 종료 날짜
path = 116
row = 35  # 대상 Landsat 장면 (예: 93/84)

In [4]:
# AOI 내의 Landsat NDVI 시계열 데이터 계산
def calc_landsat_ndvi(img):
    img_sub = img.clip(aoi)  # AOI로 이미지 자르기
    ndvi = img_sub.normalizedDifference(['SR_B5', 'SR_B4']).rename('L_ndvi')  # Landsat 8 NDVI
    Q = img_sub.select('QA_PIXEL')

    # Landsat 8의 구름 마스크 생성
    cloud2BitMask = (1 << 2)
    cloud3BitMask = (1 << 3)
    cloud4BitMask = (1 << 4)
    mask = Q.bitwiseAnd(cloud2BitMask).eq(0)\
        .And(Q.bitwiseAnd(cloud3BitMask).eq(0))\
        .And(Q.bitwiseAnd(cloud4BitMask).eq(0))\
        .rename('L_cmask')  # 구름 마스크 생성

    return ndvi.addBands(mask).copyProperties(img_sub, img_sub.propertyNames())

landsat_ndvi_mask = landsat_sr.filterDate(begin_date, end_date)\
    .filterBounds(aoi)\
    .map(calc_landsat_ndvi)

landsat_ndvi = landsat_ndvi_mask.select('L_ndvi')
landsat_cmask = landsat_ndvi_mask.select('L_cmask')

In [5]:
# mod_shp를 AOI로 설정
mod_shp = aoi

# 타겟 영역(안마도)의 위치 표시 (필요한 경우 주석 해제)
# Map.addLayer(mod_shp, {}, 'Anmado AOI')
# Map.centerObject(anmado_location, 12)

# AOI 내에서 MODIS NDVI 시계열 데이터 계산
def calc_modis_ndvi(image):
    img_sub = image.clip(mod_shp)
    ndvi = img_sub.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('MOD_NDVI')
    Q = img_sub.select('State')
    masknum = getQABits(Q, 0, 2)
    mask = masknum.eq(ee.Image(0)).Or(masknum.eq(ee.Image(3)))
    return ndvi.updateMask(mask).copyProperties(img_sub, img_sub.propertyNames())

mod_ndvi = mod_sr.filterDate(begin_date, end_date)\
    .filterBounds(mod_shp)\
    .map(calc_modis_ndvi)

In [6]:
# 원본 MODIS NDVI 시계열의 결측치를 선형 보간으로 채우기
# 선형 보간 함수 구현 필요
def linear_interp(collection, frame, nodata):
    # 이미지 컬렉션 정렬
    images = collection.sort('system:time_start')
    dates = images.aggregate_array('system:time_start')
    start_date = ee.Date(dates.get(0))
    end_date = ee.Date(dates.get(-1))
    total_days = end_date.difference(start_date, 'day').round()
    interp_dates = ee.List.sequence(0, total_days.subtract(1), frame).map(
        lambda d: start_date.advance(d, 'day')
    )
    def interpolate(date):
        date = ee.Date(date)
        prev = images.filterDate(start_date, date).sort('system:time_start', False).first()
        next = images.filterDate(date, end_date).sort('system:time_start').first()
        def get_value(img):
            return ee.Image(ee.Algorithms.If(img, img, ee.Image.constant(nodata)))
        prev = get_value(prev)
        next = get_value(next)
        diff_total = next.date().difference(prev.date(), 'day')
        diff_current = date.difference(prev.date(), 'day')
        ratio = diff_current.divide(diff_total)
        interpolated = prev.add(next.subtract(prev).multiply(ratio)).set('system:time_start', date.millis())
        return interpolated
    interpolated_images = interp_dates.map(interpolate)
    return ee.ImageCollection(interpolated_images)

In [7]:
# 필요한 매개변수 설정
frame = 8 * 4
nodata = -9999
mod_ndvi_interp = linear_interp(mod_ndvi, frame, nodata)
mod_ndvi_interp0 = mod_ndvi_interp.select(['MOD_NDVI'])

In [11]:
# Savitzky–Golay 필터를 사용하여 MODIS 보간 시계열을 평활화
# SG 필터 함수 구현 필요
def sg_filter_chen(collection, list_trend_sgCoeff, list_sg_coeff):
    # 트렌드 계수 및 SG 계수 설정
    trend_sgCoeff = ee.Array(list_trend_sgCoeff)
    sg_coeff = ee.Array(list_sg_coeff)
    # 이미지 컬렉션을 리스트로 변환
    images = collection.toList(collection.size())
    n = images.length().getInfo()
    # 각 이미지에 대해 SG 필터 적용
    def apply_sg_filter(i):
        i = ee.Number(i)
        window_size = sg_coeff.length().getInfo()
        half_window = int((window_size - 1) / 2)
        # 필터링을 위한 이미지 집합 생성
        start = i.subtract(half_window)
        end = i.add(half_window)
        indices = ee.List.sequence(start, end)
        images_subset = indices.map(lambda idx: images.get(ee.Number(idx).clamp(0, n - 1)))
        imgs = ee.ImageCollection.fromImages(images_subset)
        # 이미지들을 밴드로 결합
        imgs_bands = imgs.toBands()
        # SG 필터 적용
        filtered = imgs_bands.multiply(sg_coeff).reduce(ee.Reducer.sum())
        img = ee.Image(images.get(i)).set('system:time_start', ee.Image(images.get(i)).get('system:time_start'))
        return filtered.copyProperties(img)
    filtered_images = ee.List.sequence(0, n - 1).map(apply_sg_filter)
    return ee.ImageCollection(filtered_images)

list_trend_sgCoeff = [-0.076923102,-1.4901161e-008,0.062937059,0.11188812,0.14685316,0.16783218,
    0.17482519,0.16783218,0.14685316,0.11188812,0.062937059,-1.4901161e-008,-0.076923102]
list_sg_coeff = [-0.090909064,0.060606077,0.16883118,0.23376624,0.25541127,0.23376624,
    0.16883118,0.060606077,-0.090909064]

mod_ndvi_sg = sg_filter_chen(mod_ndvi_interp0, list_trend_sgCoeff, list_sg_coeff)

EEException: Image.date: Image does not have a 'system:time_start' property.

In [ ]:
# MODIS NDVI 이미지를 bicubic 보간법을 사용하여 30m로 리샘플링
L_ref = landsat_ndvi.first()
proj = L_ref.select('L_ndvi').projection().crs()

def resample_modis(img):
    return img.multiply(1.023).subtract(0.013).float()\
        .resample('bicubic')\
        .copyProperties(img, img.propertyNames())

mod_ndvi_sg30 = mod_ndvi_sg.map(resample_modis)

# Landsat 및 MODIS NDVI 데이터 결합
combine_ndvi = landsat_ndvi.merge(mod_ndvi_sg30)
combine_ndvi0 = landsat_cmask.merge(mod_ndvi_sg30)

# 나머지 코드를 계속 변환합니다.

# 각 Landsat 이미지와 MODIS 이미지의 날짜를 가져옵니다.
startDate = ee.Date(begin_date)
l_size = landsat_ndvi.size()
m_size = mod_ndvi_sg30.size()
combinendvi_list = combine_ndvi.toList(l_size.add(m_size))
combinendvi_list0 = combine_ndvi0.toList(l_size.add(m_size))
landsatndvi_list = combinendvi_list.slice(0, l_size)
landsatmask_list = combinendvi_list0.slice(0, l_size)
modndvi_list = combinendvi_list.slice(l_size, l_size.add(m_size))

# Landsat 이미지의 날짜 리스트 생성
list_process = ee.List.sequence(0, l_size.subtract(1))
ldate = list_process.map(lambda i: ee.Number(ee.Image(landsatndvi_list.get(i)).date().difference(startDate, 'day').toInt()))

# MODIS 이미지의 날짜 리스트 생성
list_process = ee.List.sequence(0, m_size.subtract(1))
mdate = list_process.map(lambda i: ee.Number(ee.Image(modndvi_list.get(i)).date().difference(startDate, 'day')))

# Landsat 이미지와 가장 가까운 MODIS 이미지의 인덱스 찾기
mdate0 = ee.Array(mdate)
list_process = ee.List.sequence(0, l_size.subtract(1))
lm_loc = list_process.map(lambda i: mdate0.subtract(ee.Number(ldate.get(i))).abs().toList().indexOf(mdate0.subtract(ee.Number(ldate.get(i))).abs().toList().reduce(ee.Reducer.min())))

# 각 Landsat 이미지에 대응하는 MODIS 이미지 추출
def get_modndvi_pair(i):
    i = ee.Number(i)
    onenum = ee.Number(lm_loc.get(i))
    onelist = modndvi_list.get(onenum)
    return onelist

modndviPair_list = list_process.map(get_modndvi_pair)
modndviPair = ee.ImageCollection(modndviPair_list)

# 형상 매칭 및 보정을 위한 나머지 코드를 계속 변환합니다.
# 이 부분은 복잡한 연산을 포함하므로, 필요에 따라 세부적으로 구현해야 합니다.

# ...

# 최종 결과 이미지 내보내기
# SG 필터를 적용한 NDVI 시계열을 가져옵니다.
syn_series_sg_int = syn_series_sg.map(lambda img: img.multiply(10000).toInt16())

# 내보낼 이미지 준비
syn_series_sg_int_list = syn_series_sg_int.toList(syn_series_sg_int.size())
syn_series_sg_part_list = syn_series_sg_int_list.slice(0, syn_series_sg_int.size())
syn_series_sg_part = ee.ImageCollection(syn_series_sg_part_list)
syn_series_sg_part0 = syn_series_sg_part.toBands()

# 밴드 이름 재설정
bandname = syn_series_sg_part0.bandNames()
def rename_bands(i):
    namei = ee.String(i)
    namei0 = ee.String('Landsat_MOD_NDVI_').cat(namei)
    return namei0
bandname0 = bandname.map(rename_bands)
syn_series_sg_part0 = syn_series_sg_part0.rename(bandname0)

# 이미지 내보내기 설정
task = ee.batch.Export.image.toDrive(
    image=syn_series_sg_part0,
    description='synseries_sg_CIA',
    folder='EarthEngineImages',
    fileNamePrefix='synseries_sg_CIA',
    region=aoi.getInfo()['coordinates'],
    scale=30,
    crs='EPSG:32655',
    maxPixels=1e13
)

task.start()

print('이미지 내보내기 작업이 시작되었습니다.')

In [ ]:
var mod_sr = ee.ImageCollection("MODIS/006/MOD09Q1"),
    landsat_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"),
    Subarea = ee.FeatureCollection("users/Yang_Chen/CIA_subregion");

// Define the location of Anmado and create a region of interest (AOI)
var anmado_location = ee.Geometry.Point([126.028111, 35.344031]);
var aoi = anmado_location.buffer(5000).bounds();  // 5km buffer around Anmado

//---------------------------------------------------------------------------
// the GEE JavaScript code for Gap Filling and Savitzky–Golay filtering method (GF-SG)
// This script reconstructs high-quality Landsat NDVI time series of 8-day intervals.
//---------------------------------------------------------------------------

/*Extract bitpacked flags
 * image - the band to extract the bits from.
 * start - the lowest bit to extract
 * end - the highest bit to extract*/
var getQABits = function(image, start, end) {
    var pattern = 0;
    for (var i = start; i <= end; i++) {
       pattern += 1 << i;
    }
    // Return a single band image of the extracted QA bits
    return image.bitwiseAnd(pattern).rightShift(start);
};

// main procedure
var begin_date = ee.String('2014-01-01');  // Start date of processing time
var end_date = ee.String('2015-01-01');  // End date of processing time
var path = ee.Number(116);
var row = ee.Number(35);  // target Landsat scene (e.g., 93/84)

// Calculate Landsat NDVI time series data for the AOI
var landsat_ndvi_mask = landsat_sr.filterDate(begin_date, end_date)
   .filterBounds(aoi)  // Use the AOI for filtering
   .map(function(img) {
      var img_sub = img.clip(aoi);  // Clip the image to the AOI
      var ndvi = img_sub.normalizedDifference(['SR_B5', 'SR_B4']).rename('L_ndvi');  // Landsat 8 NDVI
      var Q = img_sub.select('QA_PIXEL');

      // Cloud mask for Landsat 8
      var cloud2BitMask = (1 << 2);
      var cloud3BitMask = (1 << 3);
      var cloud4BitMask = (1 << 4);
      var mask = Q.bitwiseAnd(cloud2BitMask).eq(0)
          .and(Q.bitwiseAnd(cloud3BitMask).eq(0))
          .and(Q.bitwiseAnd(cloud4BitMask).eq(0))
          .rename('L_cmask');  // Cloud mask generation

      return ndvi.addBands(mask).copyProperties(img_sub, img_sub.propertyNames());
   });

var landsat_ndvi = landsat_ndvi_mask.select('L_ndvi');
var landsat_cmask = landsat_ndvi_mask.select('L_cmask');

// Set mod_shp to the AOI instead of the Landsat geometry
var mod_shp = aoi;

// Show the location of target area (Anmado)
Map.addLayer(mod_shp, {}, 'Anmado AOI');
Map.centerObject(anmado_location, 12);

// Calculate MODIS NDVI time series data within the AOI
var mod_ndvi = mod_sr.filterDate(begin_date, end_date)
   .filterBounds(mod_shp)  // Use the AOI for MODIS data
   .map(function(image) {
      var img_sub = image.clip(mod_shp);  // Clip the image to the AOI
      var ndvi = img_sub.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('MOD_NDVI');
      var Q = img_sub.select('State');
      var masknum = getQABits(Q, 0, 2);
      var mask = masknum.eq(ee.Image(0)).or(masknum.eq(ee.Image(3)));
      return ndvi.updateMask(mask).copyProperties(img_sub, img_sub.propertyNames());
   });

// Fill the gaps in the original MODIS NDVI time series by linear interpolation
var interl_m = require('users/Yang_Chen/GF-SG:Interpolation_v1');
var frame = 8 * 4;
var nodata = -9999;
var mod_ndvi_interp = interl_m.linearInterp(mod_ndvi, frame, nodata);
var mod_ndvi_interp0 = mod_ndvi_interp.select(['MOD_NDVI_INTER']);

// Smooth the MODIS interpolation time series by the Savitzky–Golay filter
var sg_filter = require('users/Yang_Chen/GF-SG:SG_filter_v1');
var list_trend_sgCoeff = ee.List([-0.076923102,-1.4901161e-008,0.062937059,0.11188812,0.14685316,0.16783218,
0.17482519,0.16783218,0.14685316,0.11188812,0.062937059,-1.4901161e-008,-0.076923102]);
var list_sg_coeff = ee.List([-0.090909064,0.060606077,0.16883118,0.23376624,0.25541127,0.23376624,
0.16883118,0.060606077,-0.090909064]);
var mod_ndvi_sg = sg_filter.sg_filter_chen(mod_ndvi_interp0, list_trend_sgCoeff, list_sg_coeff);

// Resample MODIS NDVI images to 30m by using the bicubic interpolation method
var L_ref = landsat_ndvi.first();
var proj = ee.Image(L_ref).select('L_ndvi').projection().crs();
var mod_ndvi_sg30 = mod_ndvi_sg.map(function(img){
   return img.multiply(1.023).subtract(0.013).float().resample('bicubic')
   .copyProperties(img, img.propertyNames());
});

// Combine Landsat and MODIS NDVI data
var combine_ndvi = landsat_ndvi.merge(mod_ndvi_sg30);
var combine_ndvi0 = landsat_cmask.merge(mod_ndvi_sg30);

// Continue with the rest of the code (unchanged)
// Get the date of each Landsat image and each MODIS image, etc.
// The rest of your code will work with the newly defined mod_shp as AOI.


//Get the date of each Landsat image and each MODIS image
var startDate = ee.Date(begin_date);
var l_size = ee.Number(landsat_ndvi.size());
var m_size = ee.Number(mod_ndvi_sg30.size());
var combinendvi_list = combine_ndvi.toList(l_size.add(m_size));
var combinendvi_list0 = combine_ndvi0.toList(l_size.add(m_size));
var landsatndvi_list = combinendvi_list.slice(0,l_size);
var landsatmask_list = combinendvi_list0.slice(0,l_size);
var modndvi_list = combinendvi_list.slice(l_size,l_size.add(m_size));
var list_process = ee.List.sequence(0,l_size.subtract(1));
var ldate = list_process.map(function(i){
    i = ee.Number(i);
    var one = landsatndvi_list.get(i);
    var date = ee.Image(one).date();
    var relDoy = date.difference(startDate, 'day');
    return relDoy.toInt();});
var list_process = ee.List.sequence(0,m_size.subtract(1));
var mdate = list_process.map(function(i){
    i = ee.Number(i);
    var one = modndvi_list.get(i);
    var date = ee.Image(one).date();
    var relDoy = date.difference(startDate, 'day');
    return relDoy;});


//Get the date with both modis and landsat
var mdate0 = ee.Array(mdate);
var list_process = ee.List.sequence(0,l_size.subtract(1));
var lm_loc = list_process.map(function(i){
    i = ee.Number(i);
    var onenum = ee.Number(ldate.get(i));
    var diff = mdate0.subtract(onenum).abs().toList();
    var minloc = diff.indexOf(diff.reduce(ee.Reducer.min()));
    return minloc;});

//extract corresponding MODIS images of each Landsat image
var list_loc_old = ee.List.repeat(1,m_size);
var replace = function(current,previous){
  previous = ee.List(previous);
  var i = ee.Number(current);
  var oneloc = ee.Number(lm_loc.get(i));
  var list_loc0 = previous.set(oneloc,0);
  return list_loc0;
};
var list_loc0 = ee.List.sequence(0,l_size.subtract(1)).iterate(replace,list_loc_old);
var list_loc = ee.List(list_loc0);
var list_process = ee.List.sequence(0,l_size.subtract(1));
var modndviPair_list = list_process.map(function(i){
    i = ee.Number(i);
    var onenum = ee.Number(lm_loc.get(i));
    var onelist = modndvi_list.get(onenum);
    return onelist;});
var modndviPair = ee.ImageCollection(modndviPair_list);

//Generate adjusted MODIS NDVI time series (series after shape matching and correction)
var list_process = ee.List.sequence(0,l_size.subtract(1));
var landsatNdviPair_list = list_process.map(function(i){
  i = ee.Number(i);
  var landsatone = landsatndvi_list.get(i);
  var landsatone_img = ee.Image(landsatone);
  var maskone = landsatmask_list.get(i);
  var maskone_img = ee.Image(maskone);
  var landsatone_masked = landsatone_img.updateMask(maskone_img);  //mask cloudy pixels in Landsat out
  return landsatone_masked;});
var landsatNdviPair = ee.ImageCollection(landsatNdviPair_list);


var W0 = ee.List.repeat(1,31); var W = ee.List.repeat(W0 ,31);  //the size of local window for searching the neighboring similar MODIS pixels
var kW = ee.Kernel.fixed(31, 31, W);
var modndviPair_nei0 = modndviPair.map(function(img){
  var img_nei = img.neighborhoodToBands(kW);
  return(img_nei);
});
var modndvisg30 = ee.ImageCollection(modndvi_list);
var modndvisg30_nei = modndvisg30.map(function(img){
  var img_nei = img.neighborhoodToBands(kW);
  return(img_nei);
});

//search Landsat cloudy pixels in the neigborhood of target pixel
var modndviPair_nei_list = modndviPair_nei0.toList(l_size);
var modndviPair_nei_list = list_process.map(function(i){
  i = ee.Number(i);
  var modisone = modndviPair_nei_list.get(i);
  var modisone_img = ee.Image(modisone);
  var maskone = landsatmask_list.get(i);
  var maskone_img = ee.Image(maskone);
  var modisone_masked = modisone_img.updateMask(maskone_img);
  return modisone_masked;});
var modndviPair_nei = ee.ImageCollection(modndviPair_nei_list);

//Calculate the correlation coefficient between similar pixel and the target pixel
var modndviPair_nei_mean = modndviPair_nei.mean();
var landsatNdviPair_mean = landsatNdviPair.mean();

var modndviPair_nei_diffm = modndviPair_nei.map(function(img){
  var img_diff = img.subtract(modndviPair_nei_mean);
  return img_diff;});
var landsatNdviPair_diffm = landsatNdviPair.map(function(img){
  var img_diff = img.subtract(landsatNdviPair_mean);
  return img_diff;});

var modndviPair_nei_diffm_list = modndviPair_nei_diffm.toList(l_size);
var landsatNdviPair_diffm_diffm_list = landsatNdviPair_diffm.toList(l_size);

var modnei_landsat_diff_mul_list = list_process.map(function(i){
  i = ee.Number(i);
  var modnei = modndviPair_nei_diffm_list.get(i);
  var modnei_img = ee.Image(modnei);
  var landsat = landsatNdviPair_diffm_diffm_list.get(i);
  var landsat_img = ee.Image(landsat);
  var landsat_modnei_mul = modnei_img.multiply(landsat_img);
  return landsat_modnei_mul;});

var modnei_landsat_diff_mul = ee.ImageCollection(modnei_landsat_diff_mul_list);
var modnei_landsat_diffmul_sum = modnei_landsat_diff_mul.sum();
var modndviPair_nei_diffm_2 = modndviPair_nei_diffm.map(function(img){
  var img_2 = img.multiply(img);
  return img_2;});
var landsatNdviPair_diffm_2 = landsatNdviPair_diffm.map(function(img){
  var img_2 = img.multiply(img);
  return img_2;});
var modndviPair_nei_diffm2_sum = modndviPair_nei_diffm_2.sum();
var landsatNdviPair_diffm2_sum = landsatNdviPair_diffm_2.sum();
var modnei_landsat_diffmul_sqrt = modndviPair_nei_diffm2_sum.multiply(landsatNdviPair_diffm2_sum)
.sqrt();
var modnei_landsat_corre = modnei_landsat_diffmul_sum.divide(modnei_landsat_diffmul_sqrt);

//Get neighboring similar pixels with the correlation coefficient above 0.8
var threshold_img = ee.Image.constant(0.8);
var threshold_mask = modnei_landsat_corre.gte(threshold_img);
var modnei_landsat_corre_valid = modnei_landsat_corre.multiply(threshold_mask);
var modnei_landsat_corre_sum = modnei_landsat_corre_valid.reduce(ee.Reducer.sum());
var modnei_landsat_corre_normal = modnei_landsat_corre_valid.divide(modnei_landsat_corre_sum);
var modndviPair_match0 = modndviPair_nei0.map(function(img){
  var onemodis_w = img.multiply(modnei_landsat_corre_normal);
  var onemodis_normal = onemodis_w.reduce(ee.Reducer.sum());
  return onemodis_normal;});

var modndvisg30_match0 = modndvisg30_nei.map(function(img){
  var onemodis_w = img.multiply(modnei_landsat_corre_normal);
  var onemodis_normal = onemodis_w.reduce(ee.Reducer.sum());
  return onemodis_normal;});  //generate the reference time series of the target pixel

//For these pixels which may be difficult to generate the reference series, the reference can be obtained by averaging the reference series of the neighboring pixels.
var wm0 = ee.List.repeat(1,21); var wm = ee.List.repeat(wm0,21);  //the size of local window for searching the neighboring pixels
var thresholdmask_sum = threshold_mask.reduce(ee.Reducer.sum());
var thresimg = ee.Image.constant(0);
var valid_flag_mask = thresholdmask_sum.neq(thresimg);
var invalid_flag_mask = thresholdmask_sum.eq(thresimg);
var modndviPair_match = modndviPair_match0.map(function(img){
  var meanimg = img.reduceNeighborhood({reducer: ee.Reducer.mean(),
  kernel:ee.Kernel.fixed(21, 21, wm)});
  var newimg = img.multiply(valid_flag_mask).add(meanimg.multiply(invalid_flag_mask));
  return newimg;});
var modndvisg30_match = modndvisg30_match0.map(function(img){
  var meanimg = img.reduceNeighborhood({reducer: ee.Reducer.mean(),
  kernel:ee.Kernel.fixed(21, 21, wm)});
  var newimg = img.multiply(valid_flag_mask).add(meanimg.multiply(invalid_flag_mask));
  return newimg;});  //search corresponding MODIS time series data of clear Landsat points

//shape correction by linear transfer function
var modndviPair_match_list = modndviPair_match.toList(l_size);
var modndviPair_matchmask_list = list_process.map(function(i){
  i = ee.Number(i);
  var modisone = modndviPair_match_list.get(i);
  var modisone_img = ee.Image(modisone);
  var maskone = landsatmask_list.get(i);
  var maskone_img = ee.Image(maskone);
  var modisone_masked = modisone_img.updateMask(maskone_img);
  return modisone_masked;});

var mergelist = list_process.map(function(i){
    i = ee.Number(i);
    var one = ee.Image(landsatNdviPair_list.get(i)).rename('y');
    var one0 = ee.Image(modndviPair_matchmask_list.get(i)).rename('x');
    var two = one.addBands(one0).addBands(ee.Image.constant(1));
    return two;});
var merge = ee.ImageCollection(mergelist);
var independents = ee.List(['constant','x']);
var dependent = ee.String('y');
var trend = merge.select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1));
var coefficients = trend.select('coefficients')
.arrayProject([0])
.arrayFlatten([independents]);  //b1 and b0 are solved by minimizing the difference between adjusted time series and the corresponding Landsat time series
var b1 = coefficients.select('x');
var b0 = coefficients.select('constant');

var modndvisg30_adjust = modndvisg30_match.map(function(img){
  var newimg = img.multiply(b1).add(b0);
  return newimg;});

var combine_lm = modndvisg30_adjust.merge(ee.ImageCollection(landsatndvi_list));
var combine_lm_list = combine_lm.toList(l_size.add(m_size));
var modndvisg30_adjust_list = combine_lm_list.slice(0,m_size);
var landsatndvi_list0 = combine_lm_list.slice(m_size,l_size.add(m_size));

//filling the missing values with the corresponding values of modndvisg30_adjust
var list_process = ee.List.sequence(0,m_size.subtract(1));
var syn_series_list = list_process.map(function(i){
  i = ee.Number(i);
  var modimg = ee.Image(modndvisg30_adjust_list.get(i));
  var flag = lm_loc.indexOf(i);
  var imgd = ee.Image(modndvi_list.get(i));
  var landsatimg = ee.Image(landsatndvi_list0.get(flag));
  var landsatmask0 = ee.Image(landsatmask_list.get(flag));
  var validone = ee.Number(1).subtract(list_loc.get(i));
  var validimg = ee.Image.constant(validone);
  var landsatmask = landsatmask0.multiply(validimg);
  var relocimg = landsatimg.multiply(landsatmask).add(modimg
  .subtract(modimg.multiply(landsatmask)));
  return relocimg.rename('MOD_NDVI_INTER')
  .set('system:time_start', imgd.get('system:time_start'));});


var syn_series = ee.ImageCollection(syn_series_list);

//Reduce the residual noise in the synthesized NDVI time series by the weighted SG filter
var list_trend_sgCoeff = ee.List([-0.070588261,-0.011764720,0.038009040,0.078733027,0.11040724,0.13303168,0.14660634,
0.15113123,0.14660634,0.13303168,0.11040724,0.078733027,0.038009040,-0.011764720,-0.070588261]);   //suggested trend parameter:(7,7,0,2)
var list_sg_coeff = ee.List([0.034965038,-0.12820521,0.069930017,0.31468537,0.41724950,0.31468537,
0.069930017,-0.12820521,0.034965038]);   //suggested parameters of SG:(4,4,0,5)
var syn_series_sg = sg_filter.sg_filter_chen(syn_series,list_trend_sgCoeff,list_sg_coeff);

var syn_series_sg_int = syn_series_sg.map(function(img){
  var constant = ee.Image.constant(10000);
  var newimg = img.multiply(constant).toInt16();
  return newimg});

print(syn_series_sg_int);

//output procedure
//choose date range (the whole (0,46) or part of time series (10,15)) to export
var syn_series_sg_int_list = syn_series_sg_int.toList(46);
var syn_series_sg_part_list = syn_series_sg_int_list.slice(0,46); //the whole (0,46) or part of time series (10,15)->(DOY81-113)
var syn_series_sg_part = ee.ImageCollection(syn_series_sg_part_list);
var syn_series_sg_part0 = syn_series_sg_part.toBands();
//rename the bands of output file
var bandname = syn_series_sg_part0.bandNames();
var bandname0 = bandname.map(function(i){
  var namei = ee.String(i);
  var namei0 = ee.String('Landsat_MOD_NDVI_').cat(namei);
  return namei0;
});
syn_series_sg_part0 = syn_series_sg_part0.rename(bandname0);
print(syn_series_sg_part0);

//export data to google drive(set the correct crs)
Export.image.toDrive({
   image:syn_series_sg_part0,
   description: "synseries_sg_CIA",  //the name of output
   region:mod_shp,
   //region:Subarea,  //you can choose the sub area of one scene
   scale:30,
   crs:"EPSG:32655",
   maxPixels:1e13
});

//export to your own asset
/*Export.image.toAsset({
   image:syn_series_sg_part0,
   description: "synseries_CIA",
   assetId:"synseries_sg_CIA",
   region:mod_shp,
   scale:30,
   crs:"EPSG:32655",
   maxPixels:1e13
});*/